In [1]:
import time
import ctypes
import pyvisa

# Load the DLL
dll = ctypes.cdll.LoadLibrary("C:\\Program Files\\ConSys\\CSAPI.dll")

dll.RegisterParameterStringEx1.restype = ctypes.c_long
dll.RegisterParameterStringEx1.argtypes = [ctypes.c_char_p, ctypes.c_int, ctypes.c_ushort]

dll.DeRegister.restype = None
dll.DeRegister.argtypes = [ctypes.c_long]

dll.SetValue.restype = None
dll.SetValue.argtypes = [ctypes.c_long, ctypes.c_long, ctypes.c_double]

dll.GetValue.restype = ctypes.c_double
dll.GetValue.argtypes = [ctypes.c_long, ctypes.c_long]

In [14]:
consys_parameters_string = {"RF":  "PAUL1_RFAmpqopt.cU"}
consys_parameters_id = {"RF":  0}
param_string = bytes(consys_parameters_string['RF'],'utf-8')
lshandle = dll.RegisterParameterStringEx1(param_string, len(param_string),0)
# dll.SetValue(lshandle, consys_parameters_id['RF'], 50)

In [3]:
rm = pyvisa.ResourceManager()
rm.list_resources()
inst = rm.open_resource('USB0::0x1AB1::0x0610::HDO4A244801517::INSTR')
inst.query("*IDN?")
filename = "scope_log_20240930_RF_2VDiv_.txt"

with open(filename, "w") as file:
    file.write(f"time.time(), c1pp, vset\n") # c1rms, c3rms, c1pp, c3pp\n")

vMin = 299
maxVSet = 600
VSet = vMin
dll.SetValue(lshandle, consys_parameters_id['RF'], vMin)
time.sleep(1)

while VSet<maxVSet:
    with open(filename, "a") as file:
        # c1rms = inst.query(":MEASure:ITEM? VRMS,CHANnel1 ").strip()
        #c3rms = inst.query(":MEASure:ITEM? VRMS,CHANnel3 ").strip()

        #c1pp = inst.query(":MEASure:ITEM? VPP,CHANnel1 ").strip()
        #c3pp = inst.query(":MEASure:ITEM? VPP,CHANnel3 ").strip()
        dll.SetValue(lshandle, consys_parameters_id['RF'], VSet)
        counter = 0
        while counter<10:
            c1pp = inst.query(":MEASure:ITEM? VPP,CHANnel1 ").strip()

            file.write(f"{time.time()}, {c1pp}, {VSet}\n") #{c1rms}, {c3rms}, {c1pp}, {c3pp}\n")
            counter +=1
            time.sleep(0.1)
        VSet+=1
        time.sleep(1)


('USB0::0x1AB1::0x0610::HDO4A244801517::INSTR',
 'TCPIP0::cryo-rf-2chan.local::inst0::INSTR',
 'TCPIP0::cryo-scope.local::inst0::INSTR',
 'TCPIP0::10.28.1.202::hislip0::INSTR',
 'TCPIP0::10.28.1.202::inst0::INSTR',
 'ASRL3::INSTR',
 'ASRL4::INSTR')

In [9]:
dll.DeRegister(lshandle)

In [10]:
import platform

bitness, _ = platform.architecture()

print("Python interpreter bitness:", bitness)

Python interpreter bitness: 64bit
